In [ ]:
%%javascript
require("notebook/js/notebook").Notebook.prototype.scroll_to_bottom = function () {}

In [ ]:
from notebook_prelude import *

## Datasets statistics (Text and Graph)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

trans = NxGraphToTupleTransformer()

def get_graph_type(dataset, graph_type = TYPE_CONCEPT_MAP):
    files = [x for x in dataset_helper.get_all_cached_graph_datasets(dataset) if graph_helper.get_graph_type_from_filename(x) == graph_type]
    return files


LIMIT_DATASET = ['ng20']
#LIMIT_DATASET = ['ling-spam']
#LIMIT_DATASET = None
filtered_datasets = []
for dataset in dataset_helper.get_all_available_dataset_names():
    if LIMIT_DATASET and dataset not in LIMIT_DATASET: continue
    concept_maps = [x for x in get_graph_type(dataset, TYPE_CONCEPT_MAP) if 'v2' in x]
    cooccurence_graphs = [x for x in get_graph_type(dataset, TYPE_COOCCURRENCE) if '_cooccurrence_1' in x]
    if not len(concept_maps) or not len(cooccurence_graphs): continue
    filtered_datasets.append((dataset, concept_maps[0], cooccurence_graphs[0]))

data = collections.defaultdict(lambda: [])
for dataset, concept_map, cooccurence_graph in helper.log_progress(filtered_datasets):
    print('Dataset: {}'.format(dataset))

    X, Y = dataset_helper.get_dataset(dataset)
    
    # Add general dataset statistics
    data['dataset'].append(dataset)
    data['num_documents'].append(len(X))
    data['num_classes'].append(len(set(Y)))

    # Add text statistics
    count_vec = CountVectorizer()
    doc_vecs = count_vec.fit_transform(X)
    
    all_words = set(count_vec.vocabulary_.keys())
    data['document_lengths'].append([len(x) for x in X])
    data['num_words'].append(doc_vecs.sum())
    data['num_unique_words'].append(len(all_words))
    data['median_doc_length'].append(np.median([len(x) for x in X]))
    data['median_words_per_doc'].append(np.median(np.squeeze(np.asarray(doc_vecs.sum(axis = 1)))))
    
    # Add graph statistics
    for graph_type, graph_cache_file in [(TYPE_CONCEPT_MAP, concept_map), (TYPE_COOCCURRENCE, cooccurence_graph)]:
        X_graph, _ = dataset_helper.get_dataset_cached(graph_cache_file)
        trans.transform(X_graph)
        
        all_labels = []
        for _, l in X_graph:
            all_labels += l

        data['num_nodes_{}'.format(graph_type)].append(len(all_labels))
        data['num_unique_nodes_labels_{}'.format(graph_type)].append(len(set(all_labels)))
        data['num_edges_{}'.format(graph_type)].append(sum(len(adj.nonzero()[0]) for adj, labels in X_graph))

## Graph statistics, eg. ratio_edges_to_nodes

In [ ]:

df = pd.DataFrame(data).set_index('dataset')
df['ratio_unique_words'] = df.num_unique_words / df.num_nodes_cooccurrence

for graph_type in [TYPE_CONCEPT_MAP, TYPE_COOCCURRENCE]:
    df['ratio_edges_to_nodes_{}'.format(graph_type)] = df['num_edges_{}'.format(graph_type)] / df['num_nodes_{}'.format(graph_type)]
    df['ratio_nodes_to_words_{}'.format(graph_type)] = df['num_nodes_{}'.format(graph_type)] / df.num_words

    
df['ratio_unique_words'] = df.num_unique_words / df.num_nodes_cooccurrence
df['ratio_ratio_ratio_edges_to_nodes_cmap_to_coo'] = df['ratio_edges_to_nodes_concept-map'] / df['ratio_edges_to_nodes_cooccurrence']
for col in ['median_doc_length', 'median_words_per_doc']:
    df[col] = df[col].astype(int)

df = df[sorted(df.columns)]
df

### Nodes per graph type

In [ ]:
for t in GRAPH_TYPES:
    df['nodes_per_graph_{}'.format(t)] = df['num_edges_{}'.format(t)] / df.num_documents
df

### Create latex tables for text and graphs

In [ ]:
RENAME_COLS = {
    'dataset': {
        'num_documents': '# docs', 
        'classes': '# classes', 
        'median_words_per_doc': 'median #words/doc',
        'ratio_unique_words': '#uniq. words/#words' 
    },
    'graphs': {
        'ratio_nodes_to_words_concept-map': '#nodes/#words cmap',
        'ratio_nodes_to_words_cooccurrence': '#nodes/#words coo',
        'ratio_edges_to_nodes_concept-map': '#edges/#nodes cmap', 
        'ratio_edges_to_nodes_cooccurrence': '#edges/#nodes coo', 
        'nodes_per_graph_cooccurrence': '#nodes/graph cmap',
        'nodes_per_graph_concept-map': '#nodes/graph coo',
    }
}

df.loc['mean'] = df.mean()

for key, rename_cols in RENAME_COLS.items():
    df_stats = df[list(rename_cols.keys())].rename(columns = rename_cols)
    print(df_stats.to_latex(float_format = '%.2f'))

In [ ]:
with open('tmp/dataset_statistics.npy', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
with open('tmp/dataset_statistics.npy', 'rb') as f:
    df = pickle.load(f)

## Co-Occurrence graph statistics

### All-words vs. only-nouns compressino

In [ ]:
WINDOW_SIZE = 1
datasets_coo = collections.defaultdict(lambda: {})

cooccurrence_graph_files = [x for x in dataset_helper.get_all_cached_graph_datasets() if 'cooccurrence' in x]

for graph_cf in cooccurrence_graph_files:
    dataset = filename_utils.get_dataset_from_filename(graph_cf)
    window_size = filename_utils.get_cooccurrence_window_size_from_filename(graph_cf)
    words = filename_utils.get_cooccurrence_words_from_filename(graph_cf)
    if window_size != WINDOW_SIZE: continue
    if dataset in datasets_coo and words in datasets_coo[dataset]: continue
    print(graph_cf)
    X, Y = dataset_helper.get_dataset_cached(graph_cf)
    all_labels = graph_helper.get_all_node_labels(X)
    datasets_coo[dataset][words] = all_labels

In [ ]:
data = collections.defaultdict(lambda: [])
for dataset, words in datasets_coo.items():
    for word, labels in words.items():
        data['dataset'].append(dataset)
        data['word'].append(word.replace('-', '_').replace('all', 'all_words'))
        data['label_count'].append(len(labels))
    
df = pd.DataFrame(data)
df = df.pivot(index = 'dataset', columns = 'word', values = 'label_count')
df['word_ratio'] = df.only_nouns / df.all_words
df